In [11]:
import csv,sys
import math,random
import subprocess
import numpy as np
import pandas as pd
from scipy import linalg

In [12]:
class Point:
    #An point in n dimensional space
    def __init__(self, coords,label):
    #coords - A list of values, one per dimension
        self.coords = coords
        self.label = label
        self.n = len(coords)

    def __repr__(self):
        return str(self.coords)

In [13]:
class Cluster:
    #A set of points and their centroid

    def __init__(self, points):
    #points - A list of point objects

        if len(points) == 0: 
            raise Exception("ILLEGAL: empty cluster")
        # The points that belong to this cluster
        self.points = points

        # The dimensionality of the points in this cluster
        self.n = points[0].n
        self.length = 1
    
        # Assert that all points are of the same dimensionality
        for p in points:
            if p.n != self.n: raise Exception("ILLEGAL: wrong dimensions")
    
        # Set up the initial centroid (this is usually based off one point)
        self.centroid = self.calculateCentroid()

    def __repr__(self):
    #String representation of this object
        return str(self.points)

    def update(self, points):
    #Returns the distance between the previous centroid and the new after
    #recalculating and storing the new centroid.
        old_centroid = self.centroid
        self.points = points
        self.centroid = self.calculateCentroid()
        shift = getDistance(old_centroid.coords, self.centroid.coords) 
        return shift

    def calculateCentroid(self):
    #Finds a virtual center point for a group of n-dimensional points
        numPoints = len(self.points)
        # Get a list of all coordinates in this cluster
        coords = [p.coords for p in self.points]
        # Reformat that so all x's are together, all y'z etc.
        unzipped = zip(*coords)
        # Calculate the mean for each dimension
        centroid_coords = [math.fsum(dList)/numPoints for dList in unzipped]

        return Point(centroid_coords,0)


In [14]:
def getDistance(a, b):

#Euclidean distance between two n-dimensional points.
#Note: This can be very slow and does not scale well
    #if a.n != b.n:
    #    raise Exception("ILLEGAL: non comparable points")

    ret = reduce(lambda x,y: x + pow((a[y]-b[y]), 2),range(len(a)),0.0)
    return math.sqrt(ret)

In [15]:
def kmeans(points, k, cutoff):

    # Pick out k random points to use as our initial centroids
    initial = random.sample(points, k)

    # Create k clusters using those centroids
    clusters = [Cluster([p]) for p in initial]

    # Loop through the dataset until the clusters stabilize
    loopCounter = 0
    while True:
        # Create a list of lists to hold the points in each cluster
        lists = [ [] for c in clusters]
        clusterCount = len(clusters)

        # Start counting loops
        loopCounter += 1
        # For every point in the dataset ...
        for p in points:
            # Get the distance between that point and the centroid of the first
            # cluster.
            smallest_distance = getDistance(p.coords, clusters[0].centroid.coords)

            # Set the cluster this point belongs to
            clusterIndex = 0

            # For the remainder of the clusters ...
            for i in range(clusterCount - 1):
                # calculate the distance of that point to each other cluster's
                # centroid.
                distance = getDistance(p.coords, clusters[i+1].centroid.coords)
                # If it's closer to that cluster's centroid update what we
                # think the smallest distance is, and set the point to belong
                # to that cluster
                if distance < smallest_distance:
                    smallest_distance = distance
                    clusterIndex = i+1
            lists[clusterIndex].append(p)

        # Set our biggest_shift to zero for this iteration
        biggest_shift = 0.0

        # As many times as there are clusters ...
        for i in range(clusterCount):
            # Calculate how far the centroid moved in this iteration
            shift = clusters[i].update(lists[i])
            clusters[i].length = len(lists[i])
            # Keep track of the largest move from all cluster centroid updates
            biggest_shift = max(biggest_shift, shift)

        # If the centroids have stopped moving much, say we're done!
        if biggest_shift < cutoff:
            #print "Converged after %s iterations" % loopCounter
            break
    return clusters

In [16]:
def similarity_matrix(points):
    S = []
    for i in range(0, len(points)):
        row = []
        for j in range(0, len(points)):
                # scaled pairwise comparison
            if i == j:
                row.append(0)
            else:
                row.append(getDistance(points[i],points[j]))
        S.append(row)
    return S

In [17]:
def adjacency_matrix(points,knn,S):
    A = []
    if knn < len(points):
        for i in range(0, len(S)):
            A.append([])
            row = S[i][:]
            row.sort()
            row = row[:knn]
            for j in range(0, len(S[i])):
                if S[i][j] in row:
                    A[i].append(S[i][j])
                else:
                    A[i].append(0)
    else:
        A = S
    return A

In [18]:
def diag_degree_matrix(S,A):
    D = []
    for j in range(0, len(S)):
        D.append([])
        dj = 0
        index = 0
        for i in range(0, len(S[j])):
            D[j].append(0)
            if i == j:
                index = i
            dj += A[j][i]
        D[j][index] = dj
    return D

In [19]:
def main():
    data = pd.read_csv("arcene_train.data",delimiter = ' ')
    data = np.array(data)
    #np.any(np.isnan(data))
    data = list(data)
    #print data[0]
    temp = []
    for row in data:
        temp.append([int(elements) for elements in row[:10000]])
    data = temp
    labels = pd.read_csv("arcene_train.labels", delimiter = ' ')
    labels = np.array(labels)
    labels = list(labels)
    temp = []
    for row in labels:
        temp.append(int(row))
    labels = temp
    S = similarity_matrix(data)
    #print len(S[0])
    A = adjacency_matrix(data,8,S)
    #print A
    D = diag_degree_matrix(S,A)
    #print D
    temp = []
    for p in D:
        temp.append([np.array(element) for element in p])
    D = np.array(temp)
    sqrt_D = linalg.sqrtm(D)
    inv_D = linalg.inv(sqrt_D)
    invsqrt_D = linalg.sqrtm(inv_D)
    L = np.dot(np.dot(invsqrt_D,A),sqrt_D)
    #print len(L)
    #print len(L[0])
    k = 5
    eig_vals, eig_vecs = linalg.eig(L)
    eig_pairs = [((eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
    vec = np.array([ eig_pairs[i][1] for i in range(k)])
    vec = np.transpose(vec)
    vec = list(vec)
    temp = []
    for p in vec:
        temp.append([float(elements) for elements in p])
    vec = temp
    print type(vec[i])
    points = []
    for i in range(len(vec)):
        points.append(Point(vec[i],labels[i]))
    num_clusters = 2
    opt_cutoff = 0.5
    clusters = kmeans(points,num_clusters,opt_cutoff)
    accuracy = 0.0
    count_class = dict()
    for i in range(len(clusters)):
        #print (clusters[i])
        #print clusters[i].length
        for p in clusters[i].points:
            #clusters[i] = list(clusters[i])
            if p.label in count_class.keys():
                count_class[p.label]+=1
            else:
                count_class[p.label]=1
        max_count = 0
        for key in count_class:
            if count_class[key]> max_count:
                max_count = count_class[key]
        accuracy += max_count
        count_class.clear()
    accuracy = accuracy/len(data)
    print accuracy

In [20]:
main()

<type 'list'>
0.575757575758


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:44: ComplexWarning: Casting complex values to real discards the imaginary part
